In [4]:
# from platform import python_version

# print(python_version())

3.9.6


In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd




In [7]:
positive = ["This film took me by surprise. I make it a habit of finding out as little as possible about films before attending because trailers and reviews provide spoiler after spoiler. All I knew upon entering the theater is that it was a documentary about a long married couple. Filmmaker Doug Block decided to record his parents \"for posterity\" and at the beginning of the film we are treated to the requisite interviews with his parents, outspoken mother Mina, and less than forthcoming dad, Mike. I immediately found this couple interesting and had no idea where the filmmaker (Mike & Mina's son Doug) was going to take us. As a matter of fact, I doubt that Doug himself knew where he was going with this! Life takes unexpected twists and turns and this beautifully expressive film follows the journey. It is difficult to verbalize just how moved I was with this story and the unique way in which it was told. Absolutely riveting from beginning to end and it really is a must-see even if you aren't a fan of the documentary genre. This film will make you think of your own life and might even evoke memories that you thought were long forgotten. \"51 Birch Street\" is one of those rare filmgoing experiences that makes a deep impression and never leaves you.  BRAVO!!!!!!!!",
            "Although I was hoping that I'd like it a little more, this was still certainly an impressive film. There were great performances by all the leads, and the story, while not what I'd call chilling, was still effective and it kept me interested. For me, the best part of this film was the look of the picture, for it always looked cold and damp and it just really seemed to suit the film well. I also thought that the low budget suited this movie, for I don't think that a crisp picture and clear sound would have worked as well in a film this grim. All things considered, it fell a little short of my expectations, but I'm still very glad that I finally sat down to watch this movie." ]

negative = [ "While Star Trek the Motion Picture was mostly boring, Star Trek The Final Frontier is plain bad. In this terrible sequel, the crew is on shore leave when they get a distress signal from the Federation that ambassadors representing Earth, Romulus and Kronos (the Klingon home world) have been kidnapped by a renegade Vulcan bent on his quest to attain a starship to venture into the great barrier. There, he hopes to find God. This one is so bad it is hard to figure out where to begin. At the core is a good idea that is never really developed. The plot goes nowhere instead of where no man has gone before. It is almost like the writers had no idea how to end this fiasco. The action scenes don't have the suspense of Wrath of Kahn, the philosophy is boring, and the humor is stale. Now I will focus most of my anger on William Shatner. When he takes the director's chair, the ego gets bigger. Most of the focus is on him, Spock, and McCoy, but does not give the others enough to do. In any case, this is the worst of the Star Trek franchise. I should have given it three out of ten instead of five." ,
            "I have now seen quite a few films by Pedro Almodóvar, but this would have to be the most disappointing so far. This film seemed to lack the zaniness that is usually everywhere in his films, and the story just never got me interested. Many Almodóvar regulars appear in this film, so it's not like there was a lack of on-screen talent, but this film just seemed more serious than his other films. If there was a comedic edge to this movie, I certainly couldn't find it, and it made for one surprisingly weak movie."]





In [32]:
# def load_imdb_data(data_file):
#     df = pd.read_csv(data_file)
#     texts = df['review'].tolist()
#     labels = [1 if sentiment == "positive" else 0 for sentiment in df['sentiment'].tolist()]
#     return texts, labels

# data_file = "/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv"
# texts, labels = load_imdb_data(data_file)

def process_data():
    texts = [positive, negative]
    doc = {'positive' : positive, 'negative': negative}
    labels = [1 if sentiment == "positive" else 0 for sentiment in doc]
    return texts, labels

texts, labels = process_data()

In [23]:
# Set up parameters
bert_model_name = 'bert-base-uncased'
num_classes = 2
max_length = 128
batch_size = 16
num_epochs = 4
learning_rate = 2e-5

In [11]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

In [12]:
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

In [13]:
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

In [14]:
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

In [21]:
def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
        return "positive" if preds.item() == 1 else "negative"

In [26]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [27]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [30]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

/Users/anjilabudathoki/Library/Python/3.9/lib/python/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [31]:
for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train(model, train_dataloader, optimizer, scheduler, device)
        accuracy, report = evaluate(model, val_dataloader, device)
        print(f"Validation Accuracy: {accuracy:.4f}")
        print(report)

Epoch 1/4


TypeError: _forward_unimplemented() got an unexpected keyword argument 'input_ids'

In [ ]:
torch.save(model.state_dict(), "bert_classifier.pth")

In [ ]:
# Test sentiment prediction
test_text = "The movie was great and I really enjoyed the performances of the actors."
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print("The movie was great and I really enjoyed the performances of the actors.")
print(f"Predicted sentiment: {sentiment}")

In [ ]:
 # Test sentiment prediction
test_text = "The movie was so bad and I would not recommend it to anyone."
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print("The movie was so bad and I would not recommend it to anyone.")
print(f"Predicted sentiment: {sentiment}")